In [ ]:
import numpy as np
import numpy.core._methods 
import numpy.lib.format
import math
from shapely.geometry import Point, Polygon
from shapely.geometry.polygon import LinearRing, LineString

def reward_function(params):
    # Example of rewarding the agent to follow center line

    # Read input parameters
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    all_wheels_on_track = params['all_wheels_on_track']
    is_offtrack = params['is_offtrack']
    x = params['x']
    y = params['y']
    waypoints = np.asarray(params['waypoints'])
    center_line = waypoints[:,:]
    perpendicular_slope = []
    length = center_line.shape[0]
    
    for x in range(0,length):
        a = center_line[x]
        b = center_line[(x+1)%length]
        if a[0] == b[0] and a[1] == b[1]:
            a = center_line[(x-1)%length]
            b = center_line[x]
        slope = slope_division((b[1] - a[1]),(b[0] - a[0]), x, perpendicular_slope, length)
#         slope = ((a[1] - b[1])/(a[0] - b[0]))
        print("Slope for " + str(x) + " : " + str(slope))
        
        if slope is None:
            perpendicular_slope.append(0)
#             continue
        elif slope == 0:
            perpendicular_slope.append(None)
#             continue
        else:
            perpendicular_slope.append(-1*(1/slope))

    rows, cols = (center_line.shape[0], center_line.shape[1])
    distance = 0.5 * track_width
    new_outer_points = [[0 for i in range(cols)] for j in range(rows)]
    new_inner_points = [[0 for i in range(cols)] for j in range(rows)]
    
    for x in range(0, length):
        if perpendicular_slope[x] is None: 
            new_outer_points[x][0] = center_line[x][0]
            new_outer_points[x][1] = center_line[x][1]  - distance
            new_inner_points[x][0] = center_line[x][0]
            new_inner_points[x][1] = center_line[x][1]  + distance
            print("Zero detected")
        elif perpendicular_slope[x] == 0:
            new_outer_points[x][0] = center_line[x][0]  - distance
            new_outer_points[x][1] = center_line[x][1]  
            new_inner_points[x][0] = center_line[x][0]  + distance
            new_inner_points[x][1] = center_line[x][1]  
            print("None detected")
        else:    
            dx = distance / (math.sqrt((1+(perpendicular_slope[x]**2))))
            print(dx)
            dy = perpendicular_slope[x] * dx
            print(dy)
    
            new_point = [0] * 2
            new_point[0] = center_line[x][0] - dx
            new_point[1] = center_line[x][1] - dy
            if center_line[x][0] == center_line[(x+1)%length][0] and     center_line[x][1] == center_line[(x+1)%length][1]:
                print(str(x) + " AND " + str(x-1))
                pointOnLeft = isLeft(center_line[(x-1)%length],     center_line[x], new_point)
            else:
                print(str(x) + " AND " + str(x+1))
                pointOnLeft = isLeft(center_line[x], center_line[(x+1    )%length], new_point)
                
            print("IS IT LEFT : " + str(pointOnLeft))
            if (pointOnLeft):
                new_outer_points[x][0] = center_line[x][0] + dx
                new_outer_points[x][1] = center_line[x][1] + dy
                print("Left here Outside")
                new_inner_points[x][0] = new_point[0]
                new_inner_points[x][1] = new_point[1]
                print("Right here Inside")
                
            else:
                new_outer_points[x][0] = new_point[0]
                new_outer_points[x][1] = new_point[1]
                print("Right here Outside")
                new_inner_points[x][0] = center_line[x][0] + dx
                new_inner_points[x][1] = center_line[x][1] + dy
                print("Left here Inside")
            
    race_line = (center_line[:-1])  # Use this for centerline being outer bound
    LINE_ITERATIONS=1500
    for i in range(LINE_ITERATIONS):
        race_line = improve_race_line(race_line, new_inner_points, new_outer_points)
    loop_race_line = np.append(race_line, [race_line[0]], axis=0)
    
    minval = 1000
    for pair in loop_race_line:
        distance_away = ((pair[0] - x) * (pair[0] - x)) + ((pair[1] - y) *     (pair[1] - y))
        if distance_away < minval:
            minval = distance_away

    if minval <= 0.1 and (all_wheels_on_track or not is_offtrack):
        reward = 10.0
    elif minval <= 0.5 and (all_wheels_on_track or not is_offtrack):
        reward = 5.0
    elif minval <= 0.9 and (all_wheels_on_track or not is_offtrack):
        reward = 0.1
    else:
        reward = 1e-3 # likely crashed/ close to off track

    return float(reward)

def slope_division(n, d, x, perpendicular_slope, length):
    if n == 0 and d == 0 and len(perpendicular_slope) >= (x-1)%length:
        return -1 * (1/perpendicular_slope[(x-1)%length])
    return n / d if d else None

def isLeft(a, b, c):
    if a[0] == b[0] and a[1] == b[1]:
        return True
    else:
        return ((b[0] - a[0])*(c[1] - a[1]) - (b[1] - a[1])*(c[0] - a[0])) > 0;

def menger_curvature(pt1, pt2, pt3, atol=1e-3):

    vec21 = np.array([pt1[0]-pt2[0], pt1[1]-pt2[1]])
    vec23 = np.array([pt3[0]-pt2[0], pt3[1]-pt2[1]])

    norm21 = np.linalg.norm(vec21)
    norm23 = np.linalg.norm(vec23)

    theta = np.arccos(np.dot(vec21, vec23)/(norm21*norm23))
    if np.isclose(theta-np.pi, 0.0, atol=atol):
        theta = 0.0

    dist13 = np.linalg.norm(vec21-vec23)

    return 2*np.sin(theta) / dist13


def improve_race_line(old_line, inner_border, outer_border):
    XI_ITERATIONS=4

    LINE_ITERATIONS=1500
    '''Use gradient descent, inspired by K1999, to find the racing line'''
    # start with the center line
    new_line = (old_line)
    ls_inner_border = Polygon(inner_border)
    ls_outer_border = Polygon(outer_border)
    for i in range(0,len(new_line)):
        xi = new_line[i]
        npoints = len(new_line)
        prevprev = (i - 2 + npoints) % npoints
        prev = (i - 1 + npoints) % npoints
        nexxt = (i + 1 + npoints) % npoints
        nexxtnexxt = (i + 2 + npoints) % npoints
        #print("%d: %d %d %d %d %d" % (npoints, prevprev, prev, i, nexxt, nexxtnexxt))
        ci = menger_curvature(new_line[prev], xi, new_line[nexxt])
        c1 = menger_curvature(new_line[prevprev], new_line[prev], xi)
        c2 = menger_curvature(xi, new_line[nexxt], new_line[nexxtnexxt])
        target_ci = (c1 + c2) / 2
        #print("i %d ci %f target_ci %f c1 %f c2 %f" % (i, ci, target_ci, c1, c2))

        # Calculate prospective new track position, start at half-way (curvature zero)
        xi_bound1 = (xi)
        xi_bound2 = ((new_line[nexxt][0] + new_line[prev][0]) / 2.0, (new_line[nexxt][1] + new_line[prev][1]) / 2.0)
        p_xi = (xi)
        for j in range(0,XI_ITERATIONS):
            p_ci = menger_curvature(new_line[prev], p_xi, new_line[nexxt])
            #print("i: {} iter {} p_ci {} p_xi {} b1 {} b2 {}".format(i,j,p_ci,p_xi,xi_bound1, xi_bound2))
            if np.isclose(p_ci, target_ci):
                break
            if p_ci < target_ci:
                # too flat, shrinking track too much
                xi_bound2 = (p_xi)
                new_p_xi = ((xi_bound1[0] + p_xi[0]) / 2.0, (xi_bound1[1] + p_xi[1]) / 2.0)
                if Point(new_p_xi).within(ls_inner_border) or not Point(new_p_xi).within(ls_outer_border):
                    xi_bound1 = (new_p_xi)
                else:
                    p_xi = new_p_xi
            else:
                # too curved, flatten it out
                xi_bound1 = (p_xi)
                new_p_xi = ((xi_bound2[0] + p_xi[0]) / 2.0, (xi_bound2[1] + p_xi[1]) / 2.0)

                # If iteration pushes the point beyond the border of the track,
                # just abandon the refinement at this point.  As adjacent
                # points are adjusted within the track the point should gradually
                # make its way to a new position.  A better way would be to use
                # a projection of the point on the border as the new bound.  Later.
                if Point(new_p_xi).within(ls_inner_border) or not Point(new_p_xi).within(ls_outer_border):
                    xi_bound2 = (new_p_xi)
                else:
                    p_xi = new_p_xi
        new_xi = p_xi
        # New point which has mid-curvature of prev and next points but may be outside of track
        #print((new_line[i], new_xi))
        new_line[i] = new_xi
    return new_line
